# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [3]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [5]:
# Your code here; create the connection
conn = sqlite3.Connection("data/data.sqlite")

### Q.1 Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [70]:
# Your code here
Q1 = """
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM 
    customers
WHERE customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderDate = '2003-01-31'
);
"""
pd.read_sql(Q1, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


### Q.2 Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [68]:
# Your code here
Q2 = """
SELECT 
    p.productName,
    COUNT(DISTINCT od.orderNumber) AS totalOrders,
    SUM(od.quantityOrdered) AS totalItemsSold
FROM 
    products p
INNER JOIN  
    orderdetails od ON p.productCode = od.productCode
GROUP BY 
    p.productName
ORDER BY 
    totalItemsSold DESC
;
"""
pd.read_sql(Q2, conn)

,productName,totalOrders,totalItemsSold
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
...,...,...,...
104,1999 Indy 500 Monte Carlo SS,25,855
105,1911 Ford Town Car,25,832
106,1936 Mercedes Benz 500k Roadster,25,824
107,1970 Chevy Chevelle SS 454,25,803


### Q.3 Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

**A quick note on the SQL  `SELECT DISTINCT` statement:**

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [66]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
Q3 = """
SELECT 
    p.productName,
    COUNT(DISTINCT o.customerNumber) AS totalCustomers
FROM 
    products p
INNER JOIN  
    orderdetails od ON p.productCode = od.productCode
INNER JOIN  
    orders o ON od.orderNumber = o.orderNumber
WHERE 
    o.customerNumber IS NOT NULL
GROUP BY 
    p.productName
ORDER BY 
    totalCustomers DESC
;
"""
pd.read_sql(Q3, conn)

,productName,totalCustomers
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


### Q.4 Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [64]:
# Your code here
Q4 = """
SELECT DISTINCT
    e.employeeNumber,
    e.firstName,
    e.lastName,
    o.city,
    o.officeCode
FROM 
    employees e
INNER JOIN 
    customers c ON e.employeeNumber = c.salesRepEmployeeNumber
INNER JOIN  
    orders ord ON c.customerNumber = ord.customerNumber
INNER JOIN  
    orderdetails od ON ord.orderNumber = od.orderNumber
INNER JOIN  
    products p ON od.productCode = p.productCode
INNER JOIN  
    offices o ON e.officeCode = o.officeCode
WHERE p.productCode IN (
    SELECT p.productCode
    FROM products p
    JOIN orderdetails od ON p.productCode = od.productCode
    JOIN orders ord ON od.orderNumber = ord.orderNumber
    JOIN customers c ON ord.customerNumber = c.customerNumber
    GROUP BY p.productCode
    HAVING COUNT(DISTINCT c.customerNumber) < 20
);
"""
pd.read_sql(Q4, conn)

,employeeNumber,firstName,lastName,city,officeCode
0,1501,Larry,Bott,London,7
1,1166,Leslie,Thompson,San Francisco,1
2,1612,Peter,Marsh,Sydney,6
3,1611,Andy,Fixter,Sydney,6
4,1401,Pamela,Castillo,Paris,4
5,1323,George,Vanauf,NYC,3
6,1165,Leslie,Jennings,San Francisco,1
7,1216,Steve,Patterson,Boston,2
8,1286,Foon Yue,Tseng,NYC,3
9,1370,Gerard,Hernandez,Paris,4


### Q.5 Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [41]:
# Your code here
Q5 = """
SELECT 
    e.employeeNumber,
    e.firstName,
    e.lastName,
    COUNT(c.customerNumber) AS totalCustomers
FROM 
    employees e
JOIN 
    customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY 
    e.employeeNumber, e.firstName, e.lastName
HAVING 
    AVG(c.creditLimit) > 15000
ORDER BY
    totalCustomers DESC
;
"""
pd.read_sql(Q5, conn)

,employeeNumber,firstName,lastName,totalCustomers
0,1401,Pamela,Castillo,10
1,1504,Barry,Jones,9
2,1323,George,Vanauf,8
3,1501,Larry,Bott,8
4,1286,Foon Yue,Tseng,7
5,1370,Gerard,Hernandez,7
6,1165,Leslie,Jennings,6
7,1166,Leslie,Thompson,6
8,1188,Julie,Firrelli,6
9,1216,Steve,Patterson,6


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!